In [3]:
pID = 2
pID = 'sub-0' + "%02d" % (pID)

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import accuracy_score

import pickle
import scipy.io
from bci_funcs import windowed_mean, base_correct, drop_baseline
import mne

#path = '/Users/lukasgehrke/Documents/publications/2021-fastReach/data/study/eeglab2python/'
# path = 'P:\\Lukas_Gehrke\\2021-fastReach\\data\\study\\eeglab2python\\'
path = '/Volumes/projects/Lukas_Gehrke/2021-fastReach/data/study/eeglab2python/'

# Preconscious Augmentation

This script trains the classifier for a brain-computer interface that controls electrical muscle stimulation in the preconscious augmentation experiment.
The functions used to build the feature vectors are the same that are used for the online application and are found in 'bci_funcs'

A two class linear discriminant model is fitted to idle and pre-movement EEG training data. The model and a channel selection is saved.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import accuracy_score

import pickle
import scipy.io
from bci_funcs import windowed_mean, base_correct, drop_baseline
import mne

#path = '/Users/lukasgehrke/Documents/publications/2021-fastReach/data/study/eeglab2python/'
# path = 'P:\\Lukas_Gehrke\\2021-fastReach\\data\\study\\eeglab2python\\'
path = '/Volumes/projects/Lukas_Gehrke/2021-fastReach/data/study/eeglab2python/'

In [5]:
pre_move = scipy.io.loadmat(path+pID+'/pre_move.mat')
idle = scipy.io.loadmat(path+pID+'/idle.mat')
chans = np.array(pd.read_csv(path+pID+'/sel_chans.csv', header=None)).flatten()
chans = chans - 1 # Matlab to python indexing

filename = path+pID+'/chans_'+pID+'_eeg.sav'
pickle.dump(chans, open(filename, 'wb'))

windows = 10
baseline_ix = 1

idle = idle['idle'][chans,:,:]
pre_move = pre_move['pre_move'][chans,:,:]

tmp_idle_means = np.zeros((idle.shape[2], len(chans), windows))
tmp_pre_move_means = np.zeros((pre_move.shape[2], len(chans), windows))

idle_means = np.zeros((idle.shape[2], idle.shape[0] * (windows)))
pre_move_means = np.zeros((pre_move.shape[2], pre_move.shape[0] * (windows)))

baseline = 250/10 # 100 ms baseline

for trial_ix in range(0, pre_move.shape[2]):
    tmp = base_correct(pre_move[:,:,trial_ix], baseline-1)
    pre_move_means[trial_ix, :] = windowed_mean(tmp, windows = windows).flatten()

for trial_ix in range(0, idle.shape[2]):
    tmp = base_correct(idle[:,:,trial_ix], baseline-1)
    idle_means[trial_ix, :] = windowed_mean(tmp, windows = windows).flatten()

data = np.concatenate((pre_move_means, idle_means), axis = 0)

In [6]:
pre_move_class = np.ones((pre_move_means.shape[0], 1))
idle_class = np.zeros((idle_means.shape[0], 1))
classes = np.concatenate((pre_move_class, idle_class)).ravel()

In [7]:
clf = LDA(solver='eigen', shrinkage='auto')
clf.fit(data, classes)

kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
cv_results = cross_val_score(clf, data, classes, cv=kfolds)
print(cv_results.mean())
print(cv_results.std())

filename = path+pID+'/model_'+pID+'_eeg.sav'
pickle.dump(clf, open(filename, 'wb'))

0.7764492753623189
0.06092561256770983
